# Benchmark comparison

In [ ]:
import numpy as np
import pandas as pd
import data
from FitterBFGS import bfgsfitter
from scipy.optimize import curve_fit
from iminuit import Minuit

In [ ]:
from time import thread_time
from time import process_time

In [ ]:
import matplotlib.pyplot as plt

* create data for three different functions

In [ ]:
data_sin = data.testdata()
data_sin.setfuncsin()
data_exp = data.testdata()
data_exp.setfuncexp()
data_lin = data.testdata()
data_lin.setfunclin()

# Curve fit

* linear function

In [ ]:
%%capture
# number of points to fit:
pointlist = [10,100,1000,10000,100000,1000000]
nfits = 100
linlist = []
lin_parameter_list = []
lin_parameter_list_org = []
lin_parameter_list_sigma = []

for idx,el in enumerate(pointlist):

    comp_time_lin = []

    lin_fitter = bfgsfitter(data.testfunc_lin)
    data_lin.setxy(el)
    lin_parameter_list_org.append(data_lin.params)

    # bfgsfitter
    t1_start = thread_time()
    for i in range(nfits):
        p,q = lin_fitter.curve_fit(data_lin.x,data_lin.y)
        if(i == 0): 
            lin_parameter_list.append(p)
            lin_parameter_list_sigma.append(q)
    t1_stop =  thread_time() 
    comp_time_lin.append(t1_stop-t1_start)
    print(p)
    print(q)

    # second fit after initializiation
    t1_start = thread_time()
    for i in range(nfits):
        p,q = lin_fitter.curve_fit(data_lin.x,data_lin.y)
    t1_stop = thread_time()
    comp_time_lin.append(t1_stop-t1_start)

    # scipy
    t1_start = thread_time()
    for i in range(nfits):
        p,q = curve_fit(data.testfunc_lin_np,data_lin.x,data_lin.y)
    t1_stop = thread_time()
    comp_time_lin.append(t1_stop-t1_start)
    print(p)
    print(q)

    if(idx < 4):
        # minuit
        t1_start = thread_time()
        for i in range(nfits):
            def least_squares_lin(a, b):
                return sum((data_lin.y - data.testfunc_lin_np(data_lin.x, a, b)) ** 2)
            m = Minuit(least_squares_lin)
            m.migrad()
        t1_stop = thread_time()
        comp_time_lin.append(t1_stop-t1_start)
    else:
        comp_time_lin.append(0)

    linlist.append(comp_time_lin)
 

* sinus function

In [ ]:
%%capture

sinlist = []

for idx,el in enumerate(pointlist):
    comp_time_sin = []

    sin_fitter = bfgsfitter(data.testfunc_sin)
    data_sin.setxy(el)

    # bfgsfitter
    t1_start = thread_time()
    
    for i in range(nfits):
        p,q = sin_fitter.curve_fit(data_sin.x,data_sin.y)
        
    t1_stop = thread_time() 
    comp_time_sin.append(t1_stop-t1_start)
    #print(p)
    #print(q)

    # second fit after initializiation
    t1_start = thread_time()  
    for i in range(nfits):
        p,q = sin_fitter.curve_fit(data_sin.x,data_sin.y)
        
    t1_stop = thread_time() 
    comp_time_sin.append(t1_stop-t1_start)

    # scipy
    t1_start = thread_time() 
    
    for i in range(nfits): 
        p,q = curve_fit(data.testfunc_sin_np,data_sin.x,data_sin.y)
        
    t1_stop = thread_time() 
    comp_time_sin.append(t1_stop-t1_start)
    #print(p)
    #print(q)

    # minuit
    if(idx < 4):
        t1_start = thread_time()  

        for i in range(nfits):
            def least_squares_sin(a, b, c):
                return sum((data_sin.y - data.testfunc_sin_np(data_sin.x, a, b, c)) ** 2)
            m = Minuit(least_squares_sin)
            m.migrad()

        t1_stop = thread_time() 
        comp_time_sin.append(t1_stop-t1_start)
    else:
        comp_time_sin.append(0)

    
    sinlist.append(comp_time_sin)

* exponential function

In [ ]:
%%capture

explist = []

for idx,el in enumerate(pointlist):
    comp_time_exp = []
    
    exp_fitter = bfgsfitter(data.testfunc_exp)
    data_exp.setxy(el)
    # bfgsfitter
    t1_start = thread_time() 
    
    for i in range(nfits):
        p,q = exp_fitter.curve_fit(data_exp.x,data_exp.y)
        
    t1_stop = thread_time() 
    comp_time_exp.append(t1_stop-t1_start)
    #print(p)
    #print(q)

    # second fit after initializiation
    t1_start = thread_time() 
    
    for i in range(nfits):
        p,q = exp_fitter.curve_fit(data_exp.x,data_exp.y)
        
    t1_stop = thread_time() 
    comp_time_exp.append(t1_stop-t1_start)

    # scipy
    t1_start = thread_time() 
    
    for i in range(nfits):
        p,q = curve_fit(data.testfunc_exp_np,data_exp.x,data_exp.y)
        
    t1_stop = thread_time() 
    comp_time_exp.append(t1_stop-t1_start)
    #print(p)
    #print(q)
    # minuit
    if(idx < 4):
        t1_start = thread_time() 

        for i in range(nfits):
            def least_squares_exp(a, b):
                return sum((data_exp.y - data.testfunc_exp_np(data_exp.x, a, b)) ** 2)
            m = Minuit(least_squares_exp)
            m.migrad()

        t1_stop = thread_time() 
        comp_time_exp.append(t1_stop-t1_start)
    else:
        comp_time_exp.append(0)

    
    explist.append(comp_time_exp)

# plots for n_points comparison:

In [ ]:
plotlists = [linlist, sinlist, explist]
funcnames = ["linear", "sinus", "exponential"]
for idx,plotlist in enumerate(plotlists):
    print(funcnames[idx])
    ylist = np.array(plotlist)
    plt.loglog(pointlist,ylist[:,1])
    plt.plot(pointlist,ylist[:,2])
    plt.plot(pointlist[:4],ylist[:4,3])
    plt.legend(["tensorflow","scipy","iminuit"])
    plt.xlabel("number of points")
    plt.ylabel("time [s]")
    plt.savefig(funcnames[idx]+".pdf")
    plt.show()

# fit arrays 

* linear function

In [ ]:
comp_time_lin = []
array_fitter = bfgsfitter(data.testfunc_lin)


test_n = [10,100,1000,10000]
plotlist_array = []
plotlist_array_norm = []

#initial fit
xa, ya = data.return_array(1, 20, "lin")
array_fitter.curve_fit_array(xa, ya)

for el in test_n:
    
    xa, ya = data.return_array(el, 20, "lin")
    t1_start = process_time()
    array_fitter.curve_fit_array(xa, ya)
    t1_stop =  process_time()
    comp_time_lin.append(t1_stop-t1_start)
    print(t1_stop-t1_start,(t1_stop-t1_start)/el)
    plotlist_array.append(t1_stop-t1_start)
    plotlist_array_norm.append((t1_stop-t1_start)/el)

In [ ]:
plt.loglog(test_n, plotlist_array)
plt.xlabel("number of fits")
plt.ylabel("time [s]")
plt.savefig("numfits.pdf")
plt.show()
plt.loglog(test_n, plotlist_array_norm)
plt.xlabel("number of fits")
plt.ylabel("time [s]")
plt.savefig("numfits_norm.pdf")
plt.show()